In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=4

In [ ]:
import argparse
import os
import sys
import time
import numpy as np
import tensorflow as tf
from six.moves import cPickle
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
class model():
    def __init__(self,idp=1.0,profile="harmonic"):
        self.x = tf.placeholder(tf.float32, [None, 784])
        self.y = tf.placeholder(tf.int32,[None,10])
        self.idp = tf.placeholder(tf.float32,[None,1])
        self.learning_rate = 0.001
        
        self.W1 = tf.Variable(tf.random_uniform([784, 100],-0.1,0.1),name="W1")
        self.b1 = tf.Variable(tf.random_uniform([100],-0.1,0.1),name="b1")
        self.W2 = tf.Variable(tf.random_uniform([100, 100],-0.1,0.1),name="W2")
        self.b2 = tf.Variable(tf.random_uniform([100],-0.1,0.1),name="b2")
        self.W3 = tf.Variable(tf.random_uniform([100, 10],-0.1,0.1),name="W3")
        self.b3 = tf.Variable(tf.random_uniform([10],-0.1,0.1),name="b3")
        
        if profile == "linear":
            self.r1 = tf.Variable(np.linspace(1,0,num=100,endpoint=False),name="r1",dtype='float32')
            self.r2 = tf.Variable(np.linspace(1,0,num=100,endpoint=False),name="r2",dtype='float32')
            self.r3 = tf.Variable(np.linspace(1,0,num=10,endpoint=False) ,name="r3",dtype='float32')
        else:
            # profile == "harmonic":
            self.r1 = tf.Variable(1.0/(np.arange(100)+1),name="r1",dtype='float32')
            self.r2 = tf.Variable(1.0/(np.arange(100)+1),name="r2",dtype='float32')
            self.r3 = tf.Variable(1.0/(np.arange(10)+1) ,name="r3",dtype='float32')
        # t1
        n_ones = int(100 * idp)
        n_zeros = 100 - n_ones
        self.t1 = tf.Variable(np.hstack([np.ones(n_ones),np.zeros(n_zeros)]),name="t1",dtype='float32')
        
        # t2
        n_ones = int(100 * idp)
        n_zeros = 100 - n_ones
        self.t2 = tf.Variable(np.hstack([np.ones(n_ones),np.zeros(n_zeros)]),name="t2",dtype='float32')
        
        # t3
        n_ones = int(10 * idp)
        n_zeros = 10 - n_ones
        self.t3 = tf.Variable(np.hstack([np.ones(n_ones),np.zeros(n_zeros)]),name="t3",dtype='float32')
                
        self.p1 = tf.Variable(tf.multiply(self.r1,self.t1),name="p1")
        self.p2 = tf.Variable(tf.multiply(self.r2,self.t2),name="p2")
        self.p3 = tf.Variable(tf.multiply(self.r3,self.t3),name="p3")
        
        self.rm_vars = [self.r1,self.r2,self.r3,self.t1,self.t2,self.t3,self.p1,self.p2,self.p3]
            
        # calculate r1*(W1*x)+b
        z1 = tf.add(tf.multiply(self.r1,tf.matmul(self.x,self.W1)),self.b1)
        y1 = tf.nn.relu(z1)
        z2 = tf.add(tf.multiply(self.r2,tf.matmul(y1,self.W2)),self.b2)
        y2 = tf.nn.relu(z2)
        
        self.logits = tf.add(tf.multiply(self.r3,tf.matmul(y2,self.W3)),self.b3)
        
        self.probs = tf.nn.softmax(self.logits)
        
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits,labels=self.y)
        
        self.loss = tf.reduce_mean(cross_entropy)
        
        tvars = tf.trainable_variables()
        
        tvars_rm = tf.trainable_variables()
        # print(tvars_rm)
        for rm in self.rm_vars:
            tvars_rm.remove(rm)
            # print(rm)
        
        self.tvars = tvars
        self.tvars_rm = tvars_rm
        self.train_all = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss,var_list=self.tvars)
        self.train_rm_gamma = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss,var_list=self.tvars_rm)

In [ ]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
save_dir = ""
epochs = 10
batch_per_epoch = 100
batch_size = 28
config = {'epochs': epochs,
         'batch_per_epoch': batch_per_epoch,
         'batch_size': batch_size,
         'save_dir': save_dir,
         'mnist': mnist}

In [ ]:
def train(sess,model,config):
    epochs = config['epochs']
    batch_per_epoch = config['batch_per_epoch']
    batch_size = config['batch_size']
    save_dir = config['save_dir']
    mnist = config['mnist']
    #saver = tf.train.Saver(tf.global_variables())
    loss_to_draw_epoch = []
    for epoch in range(0,epochs):
        # train one epoch
        start_time = time.time()
        train_loss = []
        for batch in range(0,batch_per_epoch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _,loss = sess.run([model.train_rm_gamma,model.loss], 
                              feed_dict = {model.x:batch_x, model.y: batch_y})
            train_loss.append(loss)
            #print(loss)

        # test after one epoch

        val_loss , probs = sess.run([model.loss,model.probs],feed_dict={model.x:mnist.test.images, model.y:mnist.test.labels})

        loss_to_draw_epoch.append(val_loss)

        print(" epoch: %4d \t time: %4f \t loss: %4f \t val_loss: %4f" % 
              (epoch,time.time() - start_time,np.mean(train_loss), val_loss))

        #loss_fd.write('epoch ' + str(epoch) + ' loss ' + str(loss_val) + '\n')

    #checkpoint_path = os.path.join(save_dir, 'model.ckpt')
    #saver.save(sess, checkpoint_path, global_step=epoch)

In [ ]:
model_test = model()

In [ ]:
with tf.Session() as sess:
    for v in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
        sess.run(v.initializer)
    print("initialized")
    train(sess=sess,model=model_test,config=config)